In [1]:
!pip install datasets transformers psutil --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import time
import math
import csv
from datetime import datetime
from pathlib import Path
import torch
import torch.nn as nn
from torch.nn import functional as F
from datasets import load_dataset
from transformers import GPT2TokenizerFast
import psutil, os

In [3]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

🔧 Using device: cuda


In [4]:
batch_size = 32
block_size = 128
max_iters = 50000
eval_interval = 1000
learning_rate = 3e-4
eval_iters = 100
dropout = 0.1

# Teacher architecture (must match checkpoint)
n_embd_teacher = 256
n_head_teacher = 8
n_layer_teacher = 6

# Student architecture (smaller)
n_embd_stud = 160
n_head_stud = 4
n_layer_stud = 4

# KD hyperparameters
TEMPERATURE = 2.0
ALPHA = 0.5

print("📘 Training Configuration:")
print(f"Teacher: {n_layer_teacher} layers, {n_head_teacher} heads, {n_embd_teacher} embd")
print(f"Student: {n_layer_stud} layers, {n_head_stud} heads, {n_embd_stud} embd")
print(f"KD params: T={TEMPERATURE}, alpha={ALPHA}")


📘 Training Configuration:
Teacher: 6 layers, 8 heads, 256 embd
Student: 4 layers, 4 heads, 160 embd
KD params: T=2.0, alpha=0.5


In [5]:
print("📥 Loading WikiText-103 (50%)...")
ds = load_dataset("wikitext", "wikitext-103-raw-v1", split="train[:50%]")
print(f"✅ Loaded {len(ds)} documents")

# Preview a few samples
for i in range(2):
    print(f"\n📝 Sample {i+1}:\n{ds[i]['text'][:300]}...")


📥 Loading WikiText-103 (50%)...


README.md: 0.00B [00:00, ?B/s]

wikitext-103-raw-v1/test-00000-of-00001.(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-103-raw-v1/train-00000-of-00002(…):   0%|          | 0.00/157M [00:00<?, ?B/s]

wikitext-103-raw-v1/train-00001-of-00002(…):   0%|          | 0.00/157M [00:00<?, ?B/s]

wikitext-103-raw-v1/validation-00000-of-(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

✅ Loaded 900675 documents

📝 Sample 1:
...

📝 Sample 2:
 = Valkyria Chronicles III = 
...


In [6]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

print("🔤 Encoding dataset using GPT-2 BPE tokenizer...")
sample_text = "Transformers are powerful models for language understanding."
encoded = tokenizer.encode(sample_text)
decoded = tokenizer.decode(encoded)
print(f"🧩 Sample text:\n{sample_text}")
print(f"➡️ Encoded tokens: {encoded}")
print(f"↩️ Decoded back: {decoded}")

# Tokenize entire dataset
tokens = []
for item in ds:
    if item.get("text"):
        tokens.extend(tokenizer.encode(item["text"]))
data = torch.tensor(tokens, dtype=torch.long)

print(f"\n📈 Total tokens: {len(data):,}")
vocab_size = tokenizer.vocab_size
print(f"📚 Vocabulary size: {vocab_size}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

🔤 Encoding dataset using GPT-2 BPE tokenizer...
🧩 Sample text:
Transformers are powerful models for language understanding.
➡️ Encoded tokens: [41762, 364, 389, 3665, 4981, 329, 3303, 4547, 13]
↩️ Decoded back: Transformers are powerful models for language understanding.


Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 1024). Running this sequence through the model will result in indexing errors



📈 Total tokens: 58,984,516
📚 Vocabulary size: 50257


In [7]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

print(f"🧠 Train data tokens: {len(train_data):,}")
print(f"🧪 Validation tokens: {len(val_data):,}")

def get_batch(split):
    data_local = train_data if split == "train" else val_data
    ix = torch.randint(len(data_local) - block_size, (batch_size,))
    x = torch.stack([data_local[i:i+block_size] for i in ix])
    y = torch.stack([data_local[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

xb, yb = get_batch("train")
print(f"✅ Batch X shape: {xb.shape}, Y shape: {yb.shape}")
print(f"🔢 Example token IDs:\n{xb[0][:20].tolist()}")
print(f"🗣 Decoded snippet:\n{tokenizer.decode(xb[0][:50].tolist())}")


🧠 Train data tokens: 53,086,064
🧪 Validation tokens: 5,898,452
✅ Batch X shape: torch.Size([32, 128]), Y shape: torch.Size([32, 128])
🔢 Example token IDs:
[705, 389, 6885, 784, 8644, 1247, 38813, 411, 357, 1160, 784, 2681, 287, 1267, 890, 1267, 837, 290, 423, 257]
🗣 Decoded snippet:
 ' are 51 – 69 centimetres ( 20 – 27 in ) long ) , and have a tighter spiral . Males use their horns during rutting season to wrestle and butt heads with rivals , while females use their horns to protect their young from


In [8]:
class TeacherHead(nn.Module):
    def __init__(self, head_size, n_embd):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class TeacherMultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.heads = nn.ModuleList([TeacherHead(head_size, n_embd) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class TeacherFeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)

class TeacherBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = TeacherMultiHeadAttention(n_head, head_size, n_embd)
        self.ffwd = TeacherFeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TeacherGPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd_teacher)
        self.position_embedding_table = nn.Embedding(block_size, n_embd_teacher)
        self.blocks = nn.Sequential(
            *[TeacherBlock(n_embd_teacher, n_head_teacher) for _ in range(n_layer_teacher)]
        )
        self.ln_f = nn.LayerNorm(n_embd_teacher)
        self.lm_head = nn.Linear(n_embd_teacher, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(B*T, -1), targets.view(B*T))
        return logits, loss

In [9]:
class StudentFeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)

class AFTSimpleHead(nn.Module):
    def __init__(self, head_size, n_embd):
        super().__init__()
        self.hidden_dim = head_size
        self.to_q = nn.Linear(n_embd, head_size, bias=False)
        self.to_k = nn.Linear(n_embd, head_size, bias=False)
        self.to_v = nn.Linear(n_embd, head_size, bias=False)
        self.project = nn.Linear(head_size, head_size)
    
    def forward(self, x):
        B, T, _ = x.shape
        Q = self.to_q(x)
        K = self.to_k(x)
        V = self.to_v(x)
        
        weights = (F.softmax(K, dim=1) * V).sum(dim=1, keepdim=True)
        Q_sig = torch.sigmoid(Q)
        Yt = Q_sig * weights
        return self.project(Yt)

class AFTSimpleMultiHead(nn.Module):
    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.heads = nn.ModuleList(
            [AFTSimpleHead(head_size, n_embd) for _ in range(num_heads)]
        )
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return self.dropout(out)

class StudentBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = AFTSimpleMultiHead(n_head, head_size, n_embd)
        self.ffwd = StudentFeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class StudentGPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd_stud)
        self.position_embedding_table = nn.Embedding(block_size, n_embd_stud)
        self.blocks = nn.Sequential(
            *[StudentBlock(n_embd_stud, n_head_stud) for _ in range(n_layer_stud)]
        )
        self.ln_f = nn.LayerNorm(n_embd_stud)
        self.lm_head = nn.Linear(n_embd_stud, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(B * T, -1),
                targets.view(B * T)
            )
        return logits, loss
    
    def generate(self, idx, max_new_tokens, temperature=1.0):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [10]:
def kd_loss(student_logits, teacher_logits, targets,
            T=TEMPERATURE, alpha=ALPHA):
    """
    KD loss = alpha * hard CE + (1-alpha) * soft KL.

    student_logits: (B, T, V)
    teacher_logits: (B, T, V)
    targets      : (B, T)
    """
    B, Tseq, V = student_logits.shape

    # Hard loss: CE over all tokens (mean per token)
    ce = F.cross_entropy(
        student_logits.view(B * Tseq, V),
        targets.view(B * Tseq)
    )

    # Soft loss: KL over all tokens (mean per token)
    student_logp = F.log_softmax(
        student_logits.view(B * Tseq, V) / T, dim=-1
    )
    teacher_soft = F.softmax(
        teacher_logits.view(B * Tseq, V) / T, dim=-1
    )

    kl = F.kl_div(
        student_logp,
        teacher_soft,
        reduction="batchmean",  # now truly mean over B*T tokens
    ) * (T * T)

    return alpha * ce + (1.0 - alpha) * kl

In [11]:
# Load teacher
print("📚 Loading teacher model...")
teacher = TeacherGPT(vocab_size).to(device)
checkpoint = torch.load(
    "/kaggle/input/gpt-teacher/pytorch/default/1/best_model_teacher_.pt",
    map_location=device
)
teacher.load_state_dict(checkpoint["model_state_dict"])
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"✓ Teacher loaded: {teacher_params:,} parameters ({teacher_params/1e6:.2f}M)")

# Create student
print("🎓 Creating student model...")
model = StudentGPT(vocab_size).to(device)
student_params = sum(p.numel() for p in model.parameters())
compression_ratio = teacher_params / student_params
print(f"✓ Student created: {student_params:,} parameters ({student_params/1e6:.2f}M)")
print(f"📊 Compression ratio: {compression_ratio:.2f}x smaller")


📚 Loading teacher model...
✓ Teacher loaded: 30,549,073 parameters (30.55M)
🎓 Creating student model...
✓ Student created: 17,414,737 parameters (17.41M)
📊 Compression ratio: 1.75x smaller


In [12]:
# --------------------------------------------------------------------
# Logging and output folders
# --------------------------------------------------------------------
OUT_DIR = Path("./outputs")
OUT_DIR.mkdir(exist_ok=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

train_losses_log = []
val_losses_log = []
train_ppl_log = []
val_ppl_log = []
step_times = []
eval_steps = []
best_val_loss = float("inf")

# CSV log
log_csv = OUT_DIR / "training_metrics.csv"
with open(log_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "iteration", "train_loss", "val_loss",
        "train_perplexity", "val_perplexity",
        "step_time_s", "is_best_model", "timestamp"
    ])

def ppl_from_loss(loss_val):
    # now this is pure CE per token
    return math.exp(loss_val) if loss_val < 50 else float("inf")

# --------------------------------------------------------------------
# Evaluation with KD loss
# --------------------------------------------------------------------
@torch.no_grad()
def estimate_loss():
    model.eval()
    teacher.eval()
    out = {}
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            student_logits, _ = model(xb)
            teacher_logits, _ = teacher(xb)
            loss = kd_loss(student_logits, teacher_logits, yb)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

# --------------------------------------------------------------------
# Training Loop
# --------------------------------------------------------------------
print("\n🚀 Training begins")

global_start = time.time()
pbar = tqdm(range(max_iters), ncols=120)

for it in pbar:
    step_start = time.time()
    
    # Evaluation
    if it % eval_interval == 0:
        eval_start = time.time()
        losses = estimate_loss()
        
        train_loss = losses["train"]
        val_loss = losses["val"]
        
        train_ppl = ppl_from_loss(train_loss)
        val_ppl = ppl_from_loss(val_loss)
        
        train_losses_log.append(train_loss)
        val_losses_log.append(val_loss)
        train_ppl_log.append(train_ppl)
        val_ppl_log.append(val_ppl)
        eval_steps.append(it)
        
        eval_time = time.time() - eval_start
        
        pbar.set_description(
            f"train {train_loss:.4f} (ppl {train_ppl:.2f})  "
            f"val {val_loss:.4f} (ppl {val_ppl:.2f})"
        )
        
        # Save best model
        is_best = False
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            is_best = True
            torch.save({
                'iteration': it,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_ppl': train_ppl,
                'val_ppl': val_ppl,
            }, OUT_DIR / "best_model.pt")
            print(f"\n✓ Saved best model at iter {it} (val_loss={val_loss:.4f})")
        
        # Save metrics
        torch.save({
            "train_losses": train_losses_log,
            "val_losses": val_losses_log,
            "train_perplexities": train_ppl_log,
            "val_perplexities": val_ppl_log,
            "eval_steps": eval_steps,
            "step_times": step_times,
            "best_val_loss": best_val_loss,
        }, OUT_DIR / "training_metrics.pt")
        
        # Log to CSV
        with open(log_csv, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                it,
                f"{train_loss:.6f}", f"{val_loss:.6f}",
                f"{train_ppl:.4f}", f"{val_ppl:.4f}",
                f"{eval_time:.4f}", is_best,
                datetime.utcnow().isoformat()
            ])
    
    # Training step with KD
    xb, yb = get_batch("train")
    
    # Teacher forward pass (no gradients)
    with torch.no_grad():
        teacher_logits, _ = teacher(xb)
    
    # Student forward pass
    student_logits, _ = model(xb)
    
    # Compute KD loss
    loss = kd_loss(student_logits, teacher_logits, yb)
    
    # Backprop
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # Record timing
    step_time = time.time() - step_start
    step_times.append(step_time)

# --------------------------------------------------------------------
# Training Complete
# --------------------------------------------------------------------
total_time = time.time() - global_start
print(f"\n✅ Training complete. Total time: {total_time/60:.2f} minutes")
print(f"🏆 Best val loss: {best_val_loss:.4f}")
print(f"📈 Best val perplexity: {ppl_from_loss(best_val_loss):.2f}\n")

# Save final checkpoint
torch.save({
    'iteration': max_iters,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses_log,
    'val_losses': val_losses_log,
    'train_perplexities': train_ppl_log,
    'val_perplexities': val_ppl_log,
    'best_val_loss': best_val_loss,
}, OUT_DIR / "final_checkpoint.pt")

# --------------------------------------------------------------------
# Plotting
# --------------------------------------------------------------------
import matplotlib.pyplot as plt

data = torch.load(OUT_DIR / "training_metrics.pt")
train_losses = data["train_losses"]
val_losses = data["val_losses"]
train_ppls = data["train_perplexities"]
val_ppls = data["val_perplexities"]
eval_steps_saved = data["eval_steps"]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(eval_steps_saved, train_losses, label="Train Loss")
plt.plot(eval_steps_saved, val_losses, label="Val Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, alpha=0.3)
plt.title("Loss Curves")

plt.subplot(1, 2, 2)
plt.plot(eval_steps_saved, train_ppls, label="Train PPL")
plt.plot(eval_steps_saved, val_ppls, label="Val PPL")
plt.xlabel("Iteration")
plt.ylabel("Perplexity")
plt.legend()
plt.grid(True, alpha=0.3)
plt.title("Perplexity Curves")

plt.tight_layout()
plt.savefig(OUT_DIR / "loss_and_perplexity_curves.png", dpi=300)
plt.close()

print(f"📊 Plots saved to {OUT_DIR / 'loss_and_perplexity_curves.png'}")

# Generate summary
summary_file = OUT_DIR / "training_summary.txt"
with open(summary_file, "w") as f:
    f.write("=" * 60 + "\n")
    f.write("KNOWLEDGE DISTILLATION TRAINING SUMMARY\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Teacher: {teacher_params/1e6:.2f}M params\n")
    f.write(f"Student: {student_params/1e6:.2f}M params\n")
    f.write(f"Compression: {compression_ratio:.2f}x\n\n")
    f.write(f"Final train loss:      {train_losses[-1]:.6f}\n")
    f.write(f"Final train perplexity: {train_ppls[-1]:.4f}\n")
    f.write(f"Final val loss:        {val_losses[-1]:.6f}\n")
    f.write(f"Final val perplexity:  {val_ppls[-1]:.4f}\n\n")
    f.write(f"Best val loss:         {best_val_loss:.6f}\n")
    f.write(f"Best val perplexity:   {ppl_from_loss(best_val_loss):.4f}\n\n")
    f.write(f"Total training time:   {total_time/60:.2f} minutes\n")

print(f"📄 Summary saved to {summary_file}")


🚀 Training begins


train 9.7513 (ppl 17176.74)  val 9.7032 (ppl 16369.19):   0%|                                 | 0/50000 [00:17<?, ?it/s]


✓ Saved best model at iter 0 (val_loss=9.7032)


train 4.6378 (ppl 103.32)  val 4.6179 (ppl 101.28):   2%|▍                      | 1002/50000 [03:16<52:33:02,  3.86s/it]


✓ Saved best model at iter 1000 (val_loss=4.6179)


...


✓ Saved best model at iter 48000 (val_loss=2.8603)


train 2.7780 (ppl 16.09)  val 2.8514 (ppl 17.31):  98%|██████████████████████▌| 49001/50000 [2:26:11<1:31:22,  5.49s/it]


✓ Saved best model at iter 49000 (val_loss=2.8514)


train 2.7780 (ppl 16.09)  val 2.8514 (ppl 17.31): 100%|█████████████████████████| 50000/50000 [2:28:52<00:00,  5.60it/s]



✅ Training complete. Total time: 148.87 minutes
🏆 Best val loss: 2.8514
📈 Best val perplexity: 17.31

📊 Plots saved to outputs/loss_and_perplexity_curves.png
📄 Summary saved to outputs/training_summary.txt


In [13]:
prompt = (
    "Artificial intelligence is changing the world in profound ways. "
    "From self-driving cars to healthcare diagnostics, AI is enabling "
    "innovations that were once considered science fiction. Experts believe "
    "that the next decade will see AI integrated into nearly every aspect "
    "of daily life, transforming industries, education, and human interactions."
)

input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

out_idx = model.generate(input_ids, max_new_tokens=50, temperature=0.8)[0].tolist()
print("\n----- GENERATED TEXT -----\n")
print(tokenizer.decode(out_idx))



----- GENERATED TEXT -----

Artificial intelligence is changing the world in profound ways. From self-driving cars to healthcare diagnostics, AI is enabling innovations that were once considered science fiction. Experts believe that the next decade will see AI integrated into nearly every aspect of daily life, transforming industries, education, and human interactions.D.L. series of humans . These methods of manual methods of previously in finding freedom , use of animals are called hypotheses and development and they are the characters and personal issues of life and children and others , respectively ; and information to humans are difficult


In [14]:

torch.save(model.state_dict(), "mini_gpt.pt")

# Later, to load:
model = GPT().to(device)  
model.load_state_dict(torch.load("mini_gpt.pt", map_location=device))
model.eval()  


NameError: name 'GPT' is not defined